In [ ]:
import os
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4-1106-preview"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [ ]:
persist_directory = os.environ.get("PERSIST_DIRECTORY", "/tmp")

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_documentation",
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [ ]:
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# Build prompt
template = """Use the following pieces of context that are part of the Hummingbot Documentation to answer the user question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI(model_name=llm_name, temperature=0)
output_parser = StrOutputParser()

chain = prompt | model

In [ ]:
query = """Can you explain me how to use the pure market making strategy?
"""

In [ ]:
from langchain.schema.runnable import RunnableMap

retriever = vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke(
    input={
        "question": query,
    }
)

In [ ]:
class Strategy(BaseModel):
    """Strategy created by the LLM for the user."""
    code: str = Field(description="Provide the python code for the strategy.")
    summary: str = Field(description="Provide a concise summary of what the strategy is doing.")

In [ ]:

strategy_extraction_function = [
    convert_pydantic_to_openai_function(Strategy)
]
tagging_model = model.bind(
    functions=strategy_extraction_function,
    function_call={"name":"Strategy"}
)

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | StrOutputParser() | tagging_model | JsonOutputFunctionsParser()

In [ ]:
result = tagging_chain.invoke(
    input={
        "question": query,
    }
)

In [ ]:
print(result["summary"])

In [ ]:
print(result["code"])